<a href="https://colab.research.google.com/github/keceli/IntroductionToHPCBootcamp/blob/main/Exercises/Monday_tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Not required for Colab, since these libraries are installed by default
#!pip install torch transformers -q

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load a pre-trained model and tokenizer
model_name = "gpt2"  # You can change this to another model if you like
model_name =  "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
print(f"Tokenizer loaded: {tokenizer.name_or_path}")
print(f"Model loaded: {model.config.model_type}")
print(f"Pad token set: {tokenizer.pad_token}")
# Set the pad token for the tokenizer if it's not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Text to tokenize
text = "HPC means high-performance"
print(f"Original text: '{text}'")

# Tokenize the text
input_ids = tokenizer.encode(text, return_tensors="pt")
print(f"Token IDs: {input_ids}")

# Predict the next token
with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits[:, -1, :]  # Get the logits for the last token

# Get the top 5 predicted tokens and their probabilities
probabilities = torch.softmax(logits, dim=-1)
top_k_probabilities, top_k_indices = torch.topk(probabilities, k=5)

print("\nNext Token Predictions:")
for i in range(top_k_indices.size(1)):
    token_id = top_k_indices[0, i].item()
    probability = top_k_probabilities[0, i].item()
    predicted_token = tokenizer.decode([token_id])
    print(f"Token: '{predicted_token}', Probability: {probability:.4f}")


def generate_response(user_input):
    """
    Generates a response from the chatbot model based on user input.

    Args:
        user_input (str): The user's input string.

    Returns:
        str: The chatbot's generated response.
    """
    # Tokenize the user input
    input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt")

    # Generate a response using the model
    # Use appropriate generation parameters
    chat_history_ids = model.generate(
        input_ids,
        max_length=1000,  # Set a reasonable max length for the response
        pad_token_id=tokenizer.eos_token_id, # Use the end-of-sequence token for padding
        no_repeat_ngram_size=3, # Avoid repeating n-grams to make the response more diverse
        do_sample=True, # Enable sampling
        top_k=50, # Consider top 50 tokens
        top_p=0.95, # Use nucleus sampling
        temperature=0.7, # Control randomness
    )

    # Decode the generated response
    response = tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response

while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit", "bye"]:
        print("ChatBot: Goodbye!")
        break
    response = generate_response(user_input)
    print(f"ChatBot: {response}")


In [ ]:
# Not required for Colab, since these libraries are installed by default
#!pip install openai -q

from openai import OpenAI

# Make sure to set your OpenAI API key in your environment variables
# or replace os.getenv('OPENAI_API_KEY') with your actual key.
# In Colab, you can add it to the 'Secrets' tab on the left pane.
# Name the secret 'OPENAI_API_KEY'.

# Cola
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

client = OpenAI(api_key=OPENAI_API_KEY)

def get_openai_response(prompt, model="gpt-3.5-turbo"):
  """
  Gets a response from the OpenAI API.

  Args:
    prompt (str): The prompt to send to the model.
    model (str): The model to use (default is "gpt-3.5-turbo").

  Returns:
    str: The model's response.
  """
  try:
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content
  except Exception as e:
    return f"An error occurred: {e}"

# Example usage:
# print(get_openai_response("What is the capital of France?"))

while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit", "bye"]:
        print("ChatBot: Goodbye!")
        break
    response = get_openai_response(user_input)
    print(f"ChatBot: {response}")

In [ ]:
# You need to install langchain langchain_community
!pip install langchain langchain_community -q

In [ ]:

#!pip install langchain langchain_community -q

import os
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Make sure to set your OpenAI API key in your environment variables
# or replace os.getenv('OPENAI_API_KEY') with your actual key.
# In Colab, you can add it to the 'Secrets' tab on the left pane.
# Name the secret 'OPENAI_API_KEY'.
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Initialize the OpenAI model
llm = OpenAI(temperature=0)

# Initialize memory for the conversation
memory = ConversationBufferMemory()

# Create a conversation chain
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True # Set to True to see the prompt and response
)

# Start the conversation loop
print("Chatbot: Hi! How can I help you today?")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit", "bye"]:
        print("Chatbot: Goodbye!")
        break
    response = conversation.predict(input=user_input)
    print(f"Chatbot: {response}")

/tmp/ipython-input-2020497128.py:17: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)
/tmp/ipython-input-2020497128.py:20: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/tmp/ipython-input-2020497128.py:23: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(


Chatbot: Hi! How can I help you today?
